In [ ]:
import pandas as pd

df = pd.read_csv("/home/venkey/Downloads/fake-news/train.csv")
df


In [13]:
df.isnull().sum()


id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [14]:
df.shape

(20800, 5)

In [15]:
df = df.dropna()


In [16]:
df.shape


(18285, 5)

In [17]:
x = df.drop('label', axis=1)
x.shape


(18285, 4)

In [18]:
y= df['label']
y.shape


(18285,)

In [20]:
import tensorflow as tf


2024-07-24 21:47:19.105437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
tf.__version__

'2.13.1'

In [26]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [ ]:
voc_size = 5000
msg = x.copy()
msg.reset_index(inplace=True)
msg

In [30]:
import nltk
import re
from nltk.corpus import stopwords 

In [31]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/venkey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
## data preporcessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(msg)):
    review = re.sub('[^a-zA-Z]', ' ', msg['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
    

In [35]:
one_hot_repr = [one_hot(words, voc_size) for words in corpus]

In [39]:
sent_len = 20
embedded_docs = pad_sequences(one_hot_repr, padding='pre', maxlen=sent_len)


In [ ]:
## creating model
embedding_vector_features = 40 ## feature representation for each word index
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
print(model.summary())
              

In [42]:
import numpy as np
x_final = np.array(embedded_docs)
y_final=np.array(y)


In [44]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final,test_size=0.33, random_state=42)


In [45]:
model.fit(x_train,y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
383/383 [==============================] - 12s 25ms/step - loss: 0.2764 - accuracy: 0.8739 - val_loss: 0.1993 - val_accuracy: 0.9176
Epoch 2/10
383/383 [==============================] - 9s 22ms/step - loss: 0.1343 - accuracy: 0.9468 - val_loss: 0.1989 - val_accuracy: 0.9188
Epoch 3/10
383/383 [==============================] - 9s 23ms/step - loss: 0.0912 - accuracy: 0.9664 - val_loss: 0.2268 - val_accuracy: 0.9205
Epoch 4/10
383/383 [==============================] - 9s 22ms/step - loss: 0.0592 - accuracy: 0.9802 - val_loss: 0.2598 - val_accuracy: 0.9162
Epoch 5/10
383/383 [==============================] - 9s 23ms/step - loss: 0.0361 - accuracy: 0.9884 - val_loss: 0.3004 - val_accuracy: 0.9148
Epoch 6/10
383/383 [==============================] - 9s 23ms/step - loss: 0.0205 - accuracy: 0.9947 - val_loss: 0.3642 - val_accuracy: 0.9099
Epoch 7/10
383/383 [==============================] - 9s 23ms/step - loss: 0.0124 - accuracy: 0.9966 - val_loss: 0.5971 - val_accuracy: 0.909

In [46]:
y_pred = model.predict(x_test)
y_pred = np.where(y_pred >0.5, 1, 0)


189/189 [==============================] - 2s 8ms/step


In [48]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3053,  366],
       [ 171, 2445]])

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)


0.9110190555095278

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.89      0.92      3419
           1       0.87      0.93      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

